In [70]:
from docplex.mp.model import Model
import math

milp_model=Model(name="milp")

In [71]:
def read_data(input):
    spilt_input=input.split()
    return int(spilt_input[0])

def euclid_dist(x1,y1,x2,y2):
    return math.sqrt((x1-x2)*(x1-x2)+(y1-y2)*(y1-y2))

In [72]:
file_name="6.5.1.txt"
file_path="./data/"+file_name

f=open(file_path)

K=read_data(f.readline())
D=read_data(f.readline())
N=read_data(f.readline())

C1=[]
C2=[]

cordinate_x=[0]
cordinate_y=[0]

servingDrone=[0]
servingTruck=[0]

weight=[0]

count=1

for x in f:
    input=x.split()
    input=[float(i) for i in input]
    cordinate_x.append(input[0])
    cordinate_y.append(input[1])

    weight.append(input[2])

    if input[3]==1:
        C1.append(count)
    else:
        C2.append(count)
    
    servingTruck.append(input[4])
    servingDrone.append(input[5])

    count=count+1

cordinate_x.append(0)
cordinate_y.append(0)
servingDrone.append(0)
servingTruck.append(0)
weight.append(0)
    

In [73]:
T=12*12
delta=3600*12/T

v_max=15.557
segment=[0.7,0.4,0.6,0.7,0.8,0.9,1.0,0.7,0.6,0.5,0.7,0.8]

gammaTruck=  [[[0 for t in range (0,T)] for j in range(0,N+2)]  for i in range (0,N+2)]

cntTime=0
for t in range(0,T):
    for i in range(0,N+2):
        for j in range(0,N+2):
            if i==j:
                gammaTruck[i][j][t]=0
            else:
                dist=euclid_dist(cordinate_x[i],cordinate_y[i],cordinate_x[j],cordinate_y[j])
                currentTime=cntTime
                totalTime=0
                while dist>0:
                    if currentTime>=3600*12:
                        currentTime=currentTime%(3600*12)
                    timeLeft=3600-currentTime%3600
                    hour=math.floor(currentTime//3600)
                    if dist>=v_max*segment[hour]*timeLeft:
                        dist-=v_max*segment[hour]*timeLeft
                        totalTime=totalTime+timeLeft
                        currentTime=(hour+1)*3600
                    else:
                        totalTime=totalTime+dist/(v_max*segment[hour])
                        dist=0
                gammaTruck[i][j][t]=totalTime
    cntTime=cntTime+delta            
    
                    
            

In [74]:
type_drone=1

takeOff=0
cruise=0
land=0
cruiseAlt=0
capacity=0
energyFull=0
beta=0
gamma=0

if type_drone==1:
    takeOff=15.6464
    cruise=31.2928
    land=7.8232
    cruiseAlt=50
    capacity=2.27
    energyFull=457503
    beta=24.2
    gamma=1392
elif type_drone==2:
    takeOff=15.6464
    cruise=31.2928
    land=7.8232
    cruiseAlt=50
    capacity=2.27
    energyFull=457503
    beta=24.2
    gamma=1392
elif type_drone==3:
    takeOff=15.6464
    cruise=31.2928
    land=7.8232
    cruiseAlt=50
    capacity=2.27
    energyFull=457503
    beta=24.2
    gamma=1392
else:
    takeOff=15.6464
    cruise=31.2928
    land=7.8232
    cruiseAlt=50
    capacity=2.27
    energyFull=457503
    beta=24.2
    gamma=1392
    


In [75]:
gammaDrone= [[0 for j in range (N+2)] for i in range(N+2)]

for i in range(0,N+2):
    for j in range(0,N+2):
        if i==j:
            gammaDrone[i][j]=0
        else:
            dist=euclid_dist(cordinate_x[i],cordinate_y[i],cordinate_x[j],cordinate_y[j])
            takeOffTime=cruiseAlt/takeOff
            landingTime=cruiseAlt/land
            hoverTime=dist/cruise
            
            gammaDrone[i][j]=takeOffTime+landingTime+hoverTime

Parameter

In [76]:
B1=1000000
B2=1000000
R=10

Variables

In [77]:
x={}
for i in range (N+2):
    for j in range (N+2):
        for k in range (K+1):
            for t in range (T):
                x[i,j,k,t]=milp_model.binary_var(name=f'x_{i}_{j}_{k}_{t}')

y={}
for i in range (N+2):
    for j in range (N+2):
        for d in range (D+1):
            for r in range (R+1):
                y[i,j,d,r]=milp_model.binary_var(name=f'y_{i}_{j}_{d}_{r}')

timeFinishTruck={}

for i in range(K+1):
    timeFinishTruck[i]=milp_model.continuous_var(name=f'timeFinishTruck_{i}')

timeFinishDrone={}

for i in range(D+1):
    timeFinishDrone[i]=milp_model.continuous_var(name=f'timeFinishDrone_{i}')

timeWaitTruck={}

for i in range(K+1):
    timeWaitTruck[i]=milp_model.continuous_var(name=f'timeWaitTruck_{i}')

timeWaitDrone={}

for i in range(D+1):
    timeWaitDrone[i]=milp_model.continuous_var(name=f'timeWaitDrone_{i}')

timeVisitTruck={}
for i in range(N+2):
    for k in range(K+1):
        timeVisitTruck[i,k]=milp_model.continuous_var(name=f'timeVisitTruck_{i}_{k}')

timeVisitDrone={}
for i in range(N+2):
    for d in range(D+1):
        for r in range(R+1):
            timeVisitDrone[i,d,r]=milp_model.continuous_var(name=f'timeVisitDrone_{i}_{d}_{r}')

W={}
for i in range(N+2):
    for d in range(D+1):
        for r in range(R+1):
            W[i,d,r]=milp_model.continuous_var(name=f'W_{i}_{d}_{r}')

energy={}
for i in range(N+2):
    for d in range(D+1):
        for r in range(R+1):
            energy[i,d,r]=milp_model.continuous_var(name=f'energy_{i}_{d}_{r}')

levelTruck={}
for i in range (N+2):
    for k in range (K+1):
        levelTruck[i,k]=milp_model.continuous_var(name=f'levelTruck_{i}_{k}')


levelDrone={}
for i in range (N+2):
    for d in range (D+1):
        for r in range (R+1):
            levelDrone[i,d,r]=milp_model.continuous_var(name=f'levelDrone_{i}_{d}_{r}')

markTruck={}
for i in range (N+2):
    for k in range (K+1):
        markTruck[i,k]=milp_model.continuous_var(name=f'markTruck_{i}_{k}')

markDrone={}
for i in range (N+2):
    for d in range (D+1):
        for r in range (R+1):
            markDrone[i,d,r]=milp_model.continuous_var(name=f'markDrone_{i}_{d}_{r}')

objective

In [78]:
sumFinish=milp_model.max(milp_model.max(timeFinishTruck[k] for k in range (1,K+1)),milp_model.max(timeFinishDrone[d] for d in range (1,D+1)))
sumWait=milp_model.sum(milp_model.sum(timeWaitTruck[k] for k in range (1,K+1))+milp_model.sum(timeWaitDrone[d] for d in range (1,D+1)))

milp_model.set_multi_objective("min",[sumFinish,sumWait])

constraint

In [82]:
#8 if truck leaves depot, must return 
for k in range(1,K+1):
    milp_model.add_constraint(milp_model.sum(x[0,j,k,t] for j in range(1,N+2) for t in range (T+1))
                              ==milp_model.sum(x[i,N+1,k,t] for i in range(0,N+1) for t in range(T+1)))


#9 if truck doesn't leave depot, that truck doesn't move 
for k in range(1,K+1):
     milp_model.add(milp_model.if_then(milp_model.sum(x[0,j,k,t] for j in range (1,N+2) for t in range (T+1))==0,
                                       milp_model.sum(x[i,j,k,t] for i in range (0,N+1) for j in range (1,N+2) for t in range (T+1))==0))
     
#11 if drone leaves depot, must return 
for d in range(1,D+1):
    for r in range (1,R+1):
         milp_model.add_constraint(milp_model.sum( y[0,j,d,r]for j in C2 + [N+1] )
                              ==milp_model.sum(y[i,N+1,d,r]for i in [0]+C2))


#12 if drone doesn't leave depot in 1 trip , that drone doesn't move in that trip     

for d in range(1,D+1):
     for r in range (1,R+1):
           milp_model.add(milp_model.if_then(milp_model.sum(y[0,j,d,r] for j in C2+[N+1])==0,
                                       milp_model.sum(y[i,j,k,t] for i in [0]+C2 for j in C2+[N+1] )==0))

#14 #15 Ensure no cycle in truck route

for k in range (1,K+1):
     milp_model.add_constraint(levelTruck[0,k]==0)

for k in range (1,K+1):
     for i in range (0,N+1):
          for j in range (1,N+2):
               milp_model.add(milp_model.if_then(milp_model.sum(x[i,j,k,t] for t in range(T+1))==1,levelTruck[j,k]==levelTruck[i,k]+1))


#16 17 Ensure no cycle in drone route
for D in range (1,D+1):
     for r in range (1,R+1):
         milp_model.add_constraint(levelTruck[0,d,r]==0)

for d in range (1,D+1):
     for r in range (1,R+1):
          for i in [0]+ C2:
               for j in C2+[N+1]:
                   milp_model.add(milp_model.if_then(y[i,j,d,r]==1,levelDrone[j,d,r]==levelDrone[i,d,r]+1))

#18 1 customer that dont require truck only get visited 1 time by truck or drone

for j in C2:
     milp_model.add_constraint(milp_model.sum(x[i,j,k,t] for i in range(0,N+1) for k in range(1,K+1) for t in range(T+1))+ 
                               milp_model.sum(y[i,j,d,r] for i in [0]+C2 for d in range (1,D+1) for r in range(1,R+1))==1)

#19 For each customer that require truck, 1 truck need to visit that customer 

for j in C1:
     milp_model.add_constraint(milp_model.sum(x[i,j,k,t] for i in range(0,N+1) for k in range(1,K+1) for t in range(T+1))==1)

#20  1 customer that dont require truck ,1 truck or drone need to leave that customer

for i in C2:
     milp_model.add_constraint(milp_model.sum(x[i,j,k,t] for j in range(1,N+2) for k in range(1,K+1) for t in range(T+1))+ 
                               milp_model.sum(y[i,j,d,r] for j in [N+1]+C2 for d in range (1,D+1) for r in range(1,R+1))==1)

#21 For each customer that require truck, 1 truck need to leave that customer 

for i in C2:
     milp_model.add_constraint(milp_model.sum(x[i,j,k,t] for j in range(1,N+2) for k in range(1,K+1) for t in range(T+1))==1)

#22 23 If truck/drone visit customer, it must leave that customer

for j in range (1,N+1):
     for k in range (1,K+1):
          milp_model.add_constraint(milp_model.sum(x[i,j,k,t] for i in range(0,N+1) for t in range(T+1))==
                                    milp_model.sum(x[j,i,k,t] for i in range(1,N+2) for t in range(T+1)))

for j in C2:
     for d in range(1,D+1):
          for r in range(1,R+1):
               milp_model.add_constraint(milp_model.sum(y[i,j,d,r] for i in C2+[0])==sum(y[j,i,d,r] for i in C2+[N+1]))


#24 correct sequence of trip

for d in range(1,D+1):
     for r in range(1,R):
          milp_model.add_constraint(milp_model.sum(y[0,j,d,r] for j in C2)>=milp_model.sum(y[0,j,d,r+1] for j in C2))


#27

for k in range(1,K+1):
     for i in range(0,N+1):
          for j in range(1,N+2):
               for t in range(1,T+1):
                    milp_model.add_constraint(timeVisitTruck[j,k]-timeVisitTruck[i,k]-B1*x[i,j,k,t] 
                                              >= gammaTruck[i,j,t]+servingTruck[j]-B1)

#28

for i in range(0,N+1):
     for j in range(1,N+2):
          for k in (1,K+1):
               for t in range(1,T+1):
                    milp_model.add_constraint(timeVisitTruck[i,k]+B2*x[i,j,k,t] <= t*delta+B2)



#29

for i in range(0,N+1):
     for j in range(1,N+2):
          for k in (1,K+1):
               for t in range(1,T):
                    milp_model.add_constraint(timeVisitTruck[i,k]-t*delta*x[i,j,k,t] >=0)


#30
for k in range(1,K+1):
     milp_model.add_constraint(timeFinishTruck[k]==timeVisitTruck[N+1,k])


#31
for k in range(1,K+1):
     milp_model.add_constraint(timeWaitTruck[k]==milp_model.sum(timeFinishTruck[k]-timeVisitTruck[i,k] for i in range (1,N+1)
                                                                if markTruck[i,k]==1))


#32
for d in range(1,D+1):
     for r in range(1,R+1):
          milp_model.add_constraint(energy[0,d,r]==energyFull)

#33
for d in range(1,D+1):
     for r in range(1,R+1):
          for i in C2+[0,N+1]:
               milp_model.add_constraint(energy[i,d,r]>=0)


#34
for d in range(1,D+1):
     for r in range(1,R+1):
          for i in C2+[0,N+1]:
               milp_model.add_constraint(W[i,d,r]<=capacity)


#35
for d in range(1,D+1):
     for r in range(1,R+1):
          for i in C2+[0]:
               for j in C2+[N+1]:
                    milp_model.add(milp_model.if_then(y[i,j,d,r]==1,timeVisitDrone[j,d,r]==timeVisitDrone[i,d,r]+servingDrone[j]))


#36
for d in range(1,D+1):
     for r in range(1,R+1):
          for i in C2+[0]:
               for j in C2+[N+1]:
                    milp_model.add(milp_model.if_then(y[i,j,d,r]==1,W[j,d,r]==W[i,d,r]-weight[j]))


#37
for d in range(1,D+1):
     for r in range(1,R+1):
          for i in C2+[0]:
               for j in C2+[N+1]:
                    milp_model.add(milp_model.if_then(y[i,j,d,r]==1,energy[j,d,r]==energy[i,d,r]-(beta*W[i,d,r]+gamma)*gammaDrone[i,j]))


#38



#39
for d in range(1,D+1):
     milp_model.add_constraint(timeFinishDrone[d]==milp_model.sum(timeVisitDrone[N+1,d,r] for r in range(1,R+1)))

#40
for d in range(1,D+1):
     milp_model.add_constraint(timeWaitDrone[d]==milp_model.sum(timeFinishDrone[N+1,d,r] - timeVisitDrone[i,d,r] for r in range(1,R+1) 
                                                                for i in C2 if markDrone[i,d,r]==1))
     


KeyError: (0, 1, 1, 144)

In [ ]:
milp_model.print_information()

Model: milp
 - number of variables: 24
   - binary=4, integer=0, continuous=20
 - number of constraints: 0
   - linear=0
 - parameters: defaults
 - problem type is: MILP
